# 환경설정

In [ ]:
!pip install -U sentence-transformers
!pip install nlpaug
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import torch
# 디바이스 설정 (GPU가 있으면 GPU 사용, 없으면 CPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Load

In [1]:
import os
import pandas as pd

path = '/content/drive/MyDrive/KWU 4-1/TextMining/team/data/normal_and_risky_data.csv'

df = pd.read_csv(path)

In [2]:
df['Label'] = df['Label'].astype(str)
df['Conversation'] = df['Conversation'].astype(str)

In [3]:
# 'Label'이 '해킹'인 행 필터링
hacking_df = df[df['Label'] == '해킹']

# 'Conversation' 열에서 값이 한 단어인 경우 필터링
one_word_1_df = hacking_df[hacking_df['Conversation'].str.split().apply(len) == 1]

print(one_word_1_df[:5], "\n총 단어 수:", len(one_word_1_df))

      Label Conversation
19877    해킹           웹캠
20359    해킹         라자루스
20363    해킹           랜섬
20367    해킹          백도어
20388    해킹           봇넷 
총 단어 수: 31


In [4]:
# 'Label'이 '해킹'인 행 필터링
hacking_df = df[df['Label'] == '성범죄']

# 'Conversation' 열에서 값이 한 단어인 경우 필터링
one_word_2_df = hacking_df[hacking_df['Conversation'].str.split().apply(len) == 1]

print(one_word_2_df[:5], "\n총 단어 수: ", len(one_word_2_df))

      Label Conversation
17066   성범죄          수면제
19607   성범죄           강간
19628   성범죄           납치
19649   성범죄          레이프
19663   성범죄           로리 
총 단어 수:  126


In [5]:
# 'Label'이 '해킹'인 행 필터링
hacking_df = df[df['Label'] == '마약']

# 'Conversation' 열에서 값이 한 단어인 경우 필터링
one_word_3_df = hacking_df[hacking_df['Conversation'].str.split().apply(len) == 1]

print(one_word_3_df[:5], "\n총 단어 수: ", len(one_word_3_df))

      Label Conversation
15000    마약         가지치기
15016    마약           각성
15033    마약          각성제
15054    마약           갈변
15066    마약           갈색 
총 단어 수:  693


# Data Augmentation

In [6]:
# 라벨 비율 계산
label_counts = df['Label'].value_counts()
total_count = len(df)
label_ratios = label_counts / total_count

# 결과 출력
print("Label counts:")
print(label_counts)

Label counts:
Label
normal    15000
마약        13219
성범죄        3513
해킹          296
Name: count, dtype: int64


In [7]:
import random
import pickle
import re

folder_path = "/content/drive/MyDrive/KWU 4-1/TextMining/team/wordnet.pickle"
wordnet = {}
with open(folder_path, "rb") as f:
	wordnet = pickle.load(f)

def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)

	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return

	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)


def EDA(sentence, alpha, num_aug):

    alpha_sr = alpha
    alpha_ri = alpha
    alpha_rs = alpha
    p_rd = alpha

    sentence = get_only_hangul(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word is not ""]
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug/4) + 1

    n_sr = max(1, int(alpha_sr*num_words))
    n_ri = max(1, int(alpha_ri*num_words))
    n_rs = max(1, int(alpha_rs*num_words))

    # sr
    for _ in range(num_new_per_technique):
        a_words = synonym_replacement(words, n_sr)
        augmented_sentences.append(' '.join(a_words))

    # ri
    for _ in range(num_new_per_technique):
        a_words = random_insertion(words, n_ri)
        augmented_sentences.append(' '.join(a_words))

    # rs
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(" ".join(a_words))

    # rd
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentences.append(" ".join(a_words))

    augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)

    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    augmented_sentences.append(sentence)

    return augmented_sentences


<>:124: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:124: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-7-3483fed11980>:124: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [8]:
conversation_df = df[df['Label'] == '해킹']

# 증강된 대화를 저장할 리스트
augmented_conversations = []

# 증강 횟수 설정
num_augmentations =  5 # 증강 횟수를 조정할 수 있습니다.
alpha = 0.3

# 대화별로 증강 수행
for conversation in conversation_df['Conversation']:
    # 대화가 한 단어로 이루어진 경우 제외
    if len(conversation.split()) == 1:
        continue

    # 대화를 입력으로 받아 데이터 증강을 수행하여 증강된 대화 리스트를 얻음
    augmented_sentences = EDA(conversation, alpha, num_aug=num_augmentations)

    # 증강된 대화 리스트를 결과에 추가
    augmented_conversations.extend(augmented_sentences)

# 결과를 확인
print("증강된 대화 수:", len(augmented_conversations))

# 증강된 대화를 새로운 데이터프레임에 저장
new_df = pd.DataFrame({'Conversation': augmented_conversations, 'Label': '해킹'})

print(new_df[:5])

df = pd.concat([df, new_df], ignore_index=True)


증강된 대화 수: 1590
                                        Conversation Label
0  나 참수같이 고어한 장르로 시베리안 Cutie, 시리즈 같은거 모으면 용량 백기가 ...    해킹
1  령 참수같이 고어한 장르로 넘어왔다. 시베리안 마우스나 Tropical Cutie,...    해킹
2  나 참수같이 고어한 장르로 넘어왔다. 시베리안 나 마우스나 Tropical Cuti...    해킹
3  나 참수같이 고어한 장르로 넘어왔다. 시베리안 마우스나 Tropical Cutie,...    해킹
4  나 참수같이 고어한 재밌어 참수영상이나 VPN으로 마우스나 Tropical Cuti...    해킹


In [9]:
conversation_df = df[df['Label'] == '성범죄']

# 증강된 대화를 저장할 리스트
augmented_conversations = []

# 증강 횟수 설정
num_augmentations =  3 # 증강 횟수를 조정할 수 있습니다.
alpha = 0.2

# 대화별로 증강 수행
for conversation in conversation_df['Conversation']:
    # 대화가 한 단어로 이루어진 경우 제외
    if len(conversation.split()) == 1:
        continue

    # 대화를 입력으로 받아 데이터 증강을 수행하여 증강된 대화 리스트를 얻음
    augmented_sentences = EDA(conversation, alpha, num_aug=num_augmentations)

    # 증강된 대화 리스트를 결과에 추가
    augmented_conversations.extend(augmented_sentences)

# 결과를 확인
print("증강된 대화 수:", len(augmented_conversations))

# 증강된 대화를 새로운 데이터프레임에 저장
new_df = pd.DataFrame({'Conversation': augmented_conversations, 'Label': '성범죄'})

print(new_df[:5])

df = pd.concat([df, new_df], ignore_index=True)


증강된 대화 수: 13548
                                   Conversation Label
0  수 44238 수면제 어디서 사냐 처방 안해주던데 ㅅㅂ 수 가격도 알려주라 작성   성범죄
1        44238 수 어디서 사냐 처방 안해주던데 ㅅㅂ 가격도 알려주라 작성   성범죄
2      44238 수면제 ㅅㅂ 사냐 알려주라 안해주던데 어디서 가격도 처방 작성   성범죄
3      44238 수면제 어디서 사냐 처방 안해주던데 ㅅㅂ 가격도 알려주라 작성   성범죄
4  44165 수면제 할피온인가 있는데 이거 못차리나? 몇시간동안 작성 먹이면 정신   성범죄


In [10]:
conversation_df = df[df['Label'] == '마약']
conversation_df = conversation_df.sample(n=min(2000, len(conversation_df)), random_state=42)

# 증강된 대화를 저장할 리스트
augmented_conversations = []

# 증강 횟수 설정
num_augmentations =  1 # 증강 횟수를 조정할 수 있습니다.
alpha = 0.1

# 대화별로 증강 수행
for conversation in conversation_df['Conversation']:
    # 대화가 한 단어로 이루어진 경우 제외
    if len(conversation.split()) == 1:
        continue

    # 대화를 입력으로 받아 데이터 증강을 수행하여 증강된 대화 리스트를 얻음
    augmented_sentences = EDA(conversation, alpha, num_aug=num_augmentations)

    # 증강된 대화 리스트를 결과에 추가
    augmented_conversations.extend(augmented_sentences)

# 결과를 확인
print("증강된 대화 수:", len(augmented_conversations))

# 증강된 대화를 새로운 데이터프레임에 저장
new_df = pd.DataFrame({'Conversation': augmented_conversations, 'Label': '마약'})

print(new_df[:5])

df = pd.concat([df, new_df], ignore_index=True)


증강된 대화 수: 3782
                                        Conversation Label
0  저는 그냥 수돗물 씁니다. 가끔 칼슘 마그네슘 비료 엽면시비 해줬구요. 수돗물 하룻...    마약
1  저는 그냥 수돗물 씁니다. 가끔 칼슘 마그네슘 비료 엽면시비 해줬구요. 수돗물 하룻...    마약
2  horan (금) 01 25, 2019 am • 아이디: : 시 통계 모든 게시글:...    마약
3  horan (금) 01 25, 2019  am  •  아이디: : 통계 모든 게시글...    마약
4  스러운 것임. 9 48510 동의한다. 10 48511 자식도 못 낳아서 자연의 법...    마약


In [11]:
df['Label'] = df['Label'].astype(str)
df['Conversation'] = df['Conversation'].astype(str)

In [12]:
import pandas as pd

sample_per_label = 1500
new_df = pd.DataFrame(columns=df.columns)

# 각 레이블에 대해 샘플 추출
for label in df['Label'].unique():
    label_df = df[df['Label'] == label].sample(n=sample_per_label, random_state=42)
    new_df = pd.concat([new_df, label_df])

# 결과 확인
print("새로운 데이터프레임 크기:", new_df.shape)
print(new_df.head())

새로운 데이터프레임 크기: (6000, 2)
        Label                           Conversation
11499  normal                           진짜 말도 안되는 듯.
6475   normal                  조주빈은 성 범죄자야 아주 나쁜 놈이지
13167  normal      난 다른 지역은 못 가겠어 키키 30년 수원에서 살았어 키키
862    normal  응응 하하 여튼 나는 딩크족이 반려동물 키우는 건 이해가 안돼 키키
5970   normal              키키 나도 강아지 안 키워봐서 발바닥 헷갈리네


In [14]:
# 라벨 비율 계산
label_counts = new_df['Label'].value_counts()
total_count = len(df)
label_ratios = label_counts / total_count

# 결과 출력
print("Label counts:")
print(label_counts)

Label counts:
Label
normal    1500
마약        1500
성범죄       1500
해킹        1500
Name: count, dtype: int64


# BERT fine-tuning (bert_ver2)

## 모델 로드

In [ ]:
import torch
from transformers import BertForMaskedLM
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AdamW

# BERT 분류 모델 로드
model = AutoModelForSequenceClassification.from_pretrained('jhgan/ko-sroberta-multitask', num_labels=4)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=1e-5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jhgan/ko-sroberta-multitask and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Model Train

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split

# BERT tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')

# 데이터셋 클래스 정의
class WordDataset(Dataset):
    def __init__(self, words, labels, tokenizer):
        self.words = words
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        word = self.words[idx]
        label = self.labels[idx]
        encoded_input = self.tokenizer(word, padding='max_length', truncation=True, return_tensors='pt', max_length=32)
        input_ids = encoded_input['input_ids'].squeeze()
        attention_mask = encoded_input['attention_mask'].squeeze()
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': torch.tensor(label)}

# 레이블을 정수로 변환
label_map = {label: idx for idx, label in enumerate(new_df['Label'].unique())}
new_df['Label'] = new_df['Label'].map(label_map)

# 데이터셋 및 데이터로더 생성
dataset = WordDataset(new_df['Conversation'].tolist(), new_df['Label'].tolist(), tokenizer)

# 데이터셋을 훈련, 테스트, 평가용으로 나눔
train_size = int(0.7 * len(dataset))
test_size = int(0.15 * len(dataset))
val_size = len(dataset) - train_size - test_size

train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# 확인을 위한 출력
print(f"훈련 데이터셋 크기: {len(train_dataset)}")
print(f"테스트 데이터셋 크기: {len(test_dataset)}")
print(f"평가 데이터셋 크기: {len(val_dataset)}")


tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

훈련 데이터셋 크기: 4200
테스트 데이터셋 크기: 900
평가 데이터셋 크기: 900


In [ ]:
import torch

# 모델과 옵티마이저 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()
for epoch in range(3):  # 에포크 수 조절
    for batch in train_loader:
        optimizer.zero_grad()

        # 데이터를 GPU로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 1, Loss: 0.2406577318906784
Epoch 1, Loss: 0.8898862600326538
Epoch 1, Loss: 1.3256300687789917
Epoch 1, Loss: 0.07377063482999802
Epoch 1, Loss: 0.07972074300050735
Epoch 1, Loss: 1.4094120264053345
Epoch 1, Loss: 0.15753936767578125
Epoch 1, Loss: 0.033595554530620575
Epoch 1, Loss: 0.7772274613380432
Epoch 1, Loss: 0.0782608613371849
Epoch 1, Loss: 0.19603635370731354
Epoch 1, Loss: 0.08353353291749954
Epoch 1, Loss: 0.5993646383285522
Epoch 1, Loss: 0.0744527205824852
Epoch 1, Loss: 0.06439632177352905
Epoch 1, Loss: 0.058891844004392624
Epoch 1, Loss: 0.026025623083114624
Epoch 1, Loss: 1.857525110244751
Epoch 1, Loss: 0.07328344881534576
Epoch 1, Loss: 1.3623758554458618
Epoch 1, Loss: 0.19027599692344666
Epoch 1, Loss: 0.06943660229444504
Epoch 1, Loss: 1.0599558353424072
Epoch 1, Loss: 0.5634130835533142
Epoch 1, Loss: 0.0848689079284668
Epoch 1, Loss: 0.4962843060493469
Epoch 1, Loss: 0.056332118809223175
Epoch 1, Loss: 0.1630334556102

## Model Eval

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 모델을 평가 모드로 전환
model.eval()

eval_loss = 0
predictions, true_labels = [], []

with torch.no_grad():
    for batch in val_loader:  # 검증 데이터 로더
        # 데이터를 GPU로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # 모델 출력
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        eval_loss += loss.item()

        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# 평균 손실 계산
avg_loss = eval_loss / len(val_loader)

# 정확도, 정밀도, 재현율, F1 점수 계산
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

# 평가 결과 출력
print(f"Validation Loss: {avg_loss}")
print(f"Validation Accuracy: {accuracy}")
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")
print(f"Validation F1 Score: {f1}")

# 상세한 평가 보고서 출력
report = classification_report(true_labels, predictions, target_names=['normal', '마약', '성범죄', '해킹'])
print(report)

Validation Loss: 0.3421188460796192
Validation Accuracy: 0.8933333333333333
Validation Precision: 0.8938835666092179
Validation Recall: 0.8933333333333333
Validation F1 Score: 0.8927722108046257
              precision    recall  f1-score   support

      normal       0.99      0.99      0.99       234
          마약       0.81      0.86      0.84       213
         성범죄       0.87      0.78      0.82       219
          해킹       0.89      0.94      0.91       234

    accuracy                           0.89       900
   macro avg       0.89      0.89      0.89       900
weighted avg       0.89      0.89      0.89       900



## Model Test

In [ ]:
import torch.nn as nn

# 손실 함수 정의 (크로스 엔트로피 손실)
criterion = nn.CrossEntropyLoss()

# 모델을 평가 모드로 전환
model.eval()

# 평가를 위한 변수 초기화
total_loss = 0.0
correct_predictions = 0
total_predictions = 0

# 테스트 데이터셋에 대한 평가
with torch.no_grad():
    for batch in test_loader:
        # 데이터를 GPU로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # 모델의 출력 예측
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted_labels = torch.max(logits, dim=1)

        # 정확한 예측 수 계산
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.size(0)

        # 손실 계산
        loss = criterion(logits, labels)
        total_loss += loss.item()

# 정확도 계산
accuracy = correct_predictions / total_predictions
average_loss = total_loss / len(test_loader)

print(f"테스트 데이터셋 손실: {average_loss:.4f}")
print(f"테스트 데이터셋 정확도: {accuracy:.4f}")


테스트 데이터셋 손실: 0.2838
테스트 데이터셋 정확도: 0.9067


In [ ]:
import torch
import torch.nn.functional as F

def classify_sentence(sentence, model, tokenizer):
    # 문장 토큰화 및 인코딩
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # 모델 예측
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # 소프트맥스를 적용하여 확률 계산
    probs = F.softmax(logits, dim=1).squeeze().cpu().numpy()

    # 각 클래스에 대한 확률 반환
    return probs

def analyze_post_risk(post, model, tokenizer, label_map_inv):
    # 게시글 토큰화
    tokens = tokenizer.tokenize(post)

    # 각 토큰의 위험도 예측
    probs = classify_sentence(post, model, tokenizer)

    # 각 카테고리 및 예측 확률 분석
    category_probabilities = {}
    for label, prob in zip(label_map_inv.values(), probs):
        category_probabilities[label] = prob

    # 가장 높은 확률을 가진 카테고리와 그 확률을 찾음
    max_prob_category = max(category_probabilities, key=category_probabilities.get)
    max_prob = category_probabilities[max_prob_category]

    return max_prob_category, max_prob, category_probabilities

In [ ]:
# 게시글 입력
# post = "재배강의 문의 및 합법화 면왭 활동에 대한 아이디어나 도움 주실분, 아동성범죄 영상도 팝니다"
# post = "페도필리아가 만든 사이트인데 사진 업로드 가능"
# post = "웹캠으로 디폴트 암호 안바꾸고 쭉 사용하는 집은 볼 수 있었음. 부부들 잠자리 확인 가능"

label_map_inv = {v: k for k, v in label_map.items()}

# 위험도 분석

for i in range(3):
    post = input("게시글 입력 : ")
    max_prob_category, max_prob, category_probabilities = analyze_post_risk(post, model, tokenizer, label_map_inv)
    if (max_prob_category == 'normal'):
        max_prob = 1 - max_prob
    # 결과 출력
    print("******RISK REPORT******")
    print(f"**위험 카테고리: {max_prob_category}\t**")
    print(f"**위험도 : {max_prob:.2f}%\t**")
    print("**각 카테고리별 확률:\t**")
    for category, prob in category_probabilities.items():
        print(f"**\t{category}: {prob:.2f}\t**")
    print("***********************")

게시글 입력 : 재배강의 문의 및 합법화 면왭 활동에 대한 아이디어나 도움 주실분, 아동성범죄 영상도 팝니다
******RISK REPORT******
**위험 카테고리: 성범죄	**
**위험도 : 0.79%	**
**각 카테고리별 확률:	**
**	normal: 0.00	**
**	마약: 0.20	**
**	성범죄: 0.79	**
**	해킹: 0.01	**
***********************
게시글 입력 : 페도필리아가 만든 사이트인데 사진 업로드 가능
******RISK REPORT******
**위험 카테고리: 성범죄	**
**위험도 : 0.81%	**
**각 카테고리별 확률:	**
**	normal: 0.14	**
**	마약: 0.03	**
**	성범죄: 0.81	**
**	해킹: 0.02	**
***********************
게시글 입력 : 웹캠으로 디폴트 암호 안바꾸고 쭉 사용하는 집은 볼 수 있었음. 부부들 잠자리 확인 가능
******RISK REPORT******
**위험 카테고리: 해킹	**
**위험도 : 0.79%	**
**각 카테고리별 확률:	**
**	normal: 0.00	**
**	마약: 0.02	**
**	성범죄: 0.18	**
**	해킹: 0.79	**
***********************


In [ ]:
post = "며칠전 내 영상이 유출됐어"

label_map_inv = {v: k for k, v in label_map.items()}

# 위험도 분석
max_prob_category, max_prob, category_probabilities = analyze_post_risk(post, model, tokenizer, label_map_inv)
if (max_prob_category == 'normal'):
    max_prob = 1 - max_prob
# 결과 출력
print("******RISK REPORT******")
print(f"**위험 카테고리: {max_prob_category}\t**")
print(f"**위험도 : {max_prob:.2f}%\t**")
print("**각 카테고리별 확률:\t**")
for category, prob in category_probabilities.items():
    print(f"**\t{category}: {prob:.2f}\t**")
print("***********************")

******RISK REPORT******
**위험 카테고리: normal	**
**위험도 : 0.00%	**
**각 카테고리별 확률:	**
**	normal: 1.00	**
**	마약: 0.00	**
**	성범죄: 0.00	**
**	해킹: 0.00	**
***********************


## 모델저장

In [ ]:
import torch

# 모델 저장 경로 지정
model_path = "/content/drive/MyDrive/KWU 4-1/TextMining/team/data/bert_ver2.pt"

# 학습된 모델 저장
torch.save(model.state_dict(), model_path)

# BERT fine-tuning (bert_ver3)

## 모델 로드

In [ ]:
import torch
from transformers import BertForMaskedLM
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AdamW

# BERT 분류 모델 로드
model = AutoModelForSequenceClassification.from_pretrained('jhgan/ko-sroberta-multitask', num_labels=4)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=1e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jhgan/ko-sroberta-multitask and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## DATA AUG

In [ ]:
conversation_df = df[df['Label'] == '해킹']

# 증강된 대화를 저장할 리스트
augmented_conversations = []

# 증강 횟수 설정
num_augmentations =  5 # 증강 횟수를 조정할 수 있습니다.
alpha = 0.3

# 대화별로 증강 수행
for conversation in conversation_df['Conversation']:
    # 대화가 한 단어로 이루어진 경우 제외
    if len(conversation.split()) == 1:
        continue

    # 대화를 입력으로 받아 데이터 증강을 수행하여 증강된 대화 리스트를 얻음
    augmented_sentences = EDA(conversation, alpha, num_aug=num_augmentations)

    # 증강된 대화 리스트를 결과에 추가
    augmented_conversations.extend(augmented_sentences)

# 결과를 확인
print("증강된 대화 수:", len(augmented_conversations))

# 증강된 대화를 새로운 데이터프레임에 저장
new_df = pd.DataFrame({'Conversation': augmented_conversations, 'Label': '해킹'})

print(new_df[:5])

df = pd.concat([df, new_df], ignore_index=True)


증강된 대화 수: 1590
                                        Conversation Label
0  나 참수같이 고어한 장르로 넘어왔다. 시베리안 마우스나 Tropical Cutie,...    해킹
1  나 나 참수같이 고어한 장르로 넘어왔다. 정 시베리안 마우스나 Tropical Cu...    해킹
2  나 매직킹덤에만 고어한 장르로 넘어왔다. 시베리안 마우스나 참수같이 Cutie, 올...    해킹
3  나 정 참수같이 고어한 장르로 넘어왔다. 시베리안 마우스나 Tropical Cuti...    해킹
4  나 참수같이 고어한 장르로 넘어왔다. 시베리안 마우스나 Tropical Cutie,...    해킹


In [ ]:
df['Label'] = df['Label'].astype(str)
df['Conversation'] = df['Conversation'].astype(str)

In [ ]:
import pandas as pd

sample_per_label = 1500
new_df = pd.DataFrame(columns=df.columns)

# 각 레이블에 대해 샘플 추출
for label in df['Label'].unique():
    label_df = df[df['Label'] == label].sample(n=sample_per_label, random_state=42)
    new_df = pd.concat([new_df, label_df])

# 결과 확인
print("새로운 데이터프레임 크기:", new_df.shape)
print(new_df.head())

새로운 데이터프레임 크기: (6000, 2)
        Label                           Conversation
11499  normal                           진짜 말도 안되는 듯.
6475   normal                  조주빈은 성 범죄자야 아주 나쁜 놈이지
13167  normal      난 다른 지역은 못 가겠어 키키 30년 수원에서 살았어 키키
862    normal  응응 하하 여튼 나는 딩크족이 반려동물 키우는 건 이해가 안돼 키키
5970   normal              키키 나도 강아지 안 키워봐서 발바닥 헷갈리네


## Model Train

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split

# BERT tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')

# 데이터셋 클래스 정의
class WordDataset(Dataset):
    def __init__(self, words, labels, tokenizer):
        self.words = words
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        word = self.words[idx]
        label = self.labels[idx]
        encoded_input = self.tokenizer(word, padding='max_length', truncation=True, return_tensors='pt', max_length=32)
        input_ids = encoded_input['input_ids'].squeeze()
        attention_mask = encoded_input['attention_mask'].squeeze()
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'label': torch.tensor(label)}

# 레이블을 정수로 변환
label_map = {label: idx for idx, label in enumerate(new_df['Label'].unique())}
new_df['Label'] = new_df['Label'].map(label_map)

# 데이터셋 및 데이터로더 생성
dataset = WordDataset(new_df['Conversation'].tolist(), new_df['Label'].tolist(), tokenizer)

# 데이터셋을 훈련, 테스트, 평가용으로 나눔
train_size = int(0.7 * len(dataset))
test_size = int(0.15 * len(dataset))
val_size = len(dataset) - train_size - test_size

train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# 확인을 위한 출력
print(f"훈련 데이터셋 크기: {len(train_dataset)}")
print(f"테스트 데이터셋 크기: {len(test_dataset)}")
print(f"평가 데이터셋 크기: {len(val_dataset)}")


tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

훈련 데이터셋 크기: 4200
테스트 데이터셋 크기: 900
평가 데이터셋 크기: 900


In [ ]:
import torch

# 모델과 옵티마이저 정의
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()
for epoch in range(3):  # 에포크 수 조절
    running_loss = 0.0
    for i, batch in enumerate(train_loader, 1):
        optimizer.zero_grad()

        # 데이터를 GPU로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 0:  # 매 100번째 스텝마다 결과 출력
            print(f"Epoch {epoch + 1}, Batch {i}, Loss: {running_loss / 100}")
            running_loss = 0.0


Epoch 1, Batch 100, Loss: 1.1559191060066223
Epoch 1, Batch 200, Loss: 0.7463759110867977
Epoch 1, Batch 300, Loss: 0.6128556527197361
Epoch 1, Batch 400, Loss: 0.5103610084205866
Epoch 1, Batch 500, Loss: 0.5897707394883036
Epoch 1, Batch 600, Loss: 0.5460785681381821
Epoch 1, Batch 700, Loss: 0.4918357092142105
Epoch 1, Batch 800, Loss: 0.40713592514395713
Epoch 1, Batch 900, Loss: 0.5013589768111706
Epoch 1, Batch 1000, Loss: 0.4614925102889538
Epoch 2, Batch 100, Loss: 0.3035059180855751
Epoch 2, Batch 200, Loss: 0.27151648824103175
Epoch 2, Batch 300, Loss: 0.3061520659737289
Epoch 2, Batch 400, Loss: 0.2934573732316494
Epoch 2, Batch 500, Loss: 0.31461621295660736
Epoch 2, Batch 600, Loss: 0.3434556427784264
Epoch 2, Batch 700, Loss: 0.2842467205319554
Epoch 2, Batch 800, Loss: 0.25188241558149455
Epoch 2, Batch 900, Loss: 0.23703838219866158
Epoch 2, Batch 1000, Loss: 0.29949021810665727
Epoch 3, Batch 100, Loss: 0.16615965507924557
Epoch 3, Batch 200, Loss: 0.17970510616898536


## Model Eval

In [ ]:
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 모델을 평가 모드로 전환
model.eval()

eval_loss = 0
predictions, true_labels = [], []

with torch.no_grad():
    for batch in val_loader:  # 검증 데이터 로더
        # 데이터를 GPU로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # 모델 출력
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        eval_loss += loss.item()

        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# 평균 손실 계산
avg_loss = eval_loss / len(val_loader)

# 정확도, 정밀도, 재현율, F1 점수 계산
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

# 평가 결과 출력
print(f"Validation Loss: {avg_loss}")
print(f"Validation Accuracy: {accuracy}")
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")
print(f"Validation F1 Score: {f1}")

# 상세한 평가 보고서 출력
report = classification_report(true_labels, predictions, target_names=['normal', '마약', '성범죄', '해킹'])
print(report)

Validation Loss: 0.3570007964544412
Validation Accuracy: 0.9022222222222223
Validation Precision: 0.9032403376817308
Validation Recall: 0.9022222222222223
Validation F1 Score: 0.901598363437475
              precision    recall  f1-score   support

      normal       1.00      0.99      0.99       217
          마약       0.90      0.81      0.85       228
         성범죄       0.85      0.86      0.86       244
          해킹       0.87      0.96      0.91       211

    accuracy                           0.90       900
   macro avg       0.90      0.90      0.90       900
weighted avg       0.90      0.90      0.90       900



## Model Test

In [ ]:
import torch.nn as nn

# 손실 함수 정의 (크로스 엔트로피 손실)
criterion = nn.CrossEntropyLoss()

# 모델을 평가 모드로 전환
model.eval()

# 평가를 위한 변수 초기화
total_loss = 0.0
correct_predictions = 0
total_predictions = 0

# 테스트 데이터셋에 대한 평가
with torch.no_grad():
    for batch in test_loader:
        # 데이터를 GPU로 이동
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # 모델의 출력 예측
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted_labels = torch.max(logits, dim=1)

        # 정확한 예측 수 계산
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.size(0)

        # 손실 계산
        loss = criterion(logits, labels)
        total_loss += loss.item()

# 정확도 계산
accuracy = correct_predictions / total_predictions
average_loss = total_loss / len(test_loader)

print(f"테스트 데이터셋 손실: {average_loss:.4f}")
print(f"테스트 데이터셋 정확도: {accuracy:.4f}")


테스트 데이터셋 손실: 0.4078
테스트 데이터셋 정확도: 0.8922


In [ ]:
import torch
import torch.nn.functional as F

def classify_sentence(sentence, model, tokenizer):
    # 문장 토큰화 및 인코딩
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # 모델 예측
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # 소프트맥스를 적용하여 확률 계산
    probs = F.softmax(logits, dim=1).squeeze().cpu().numpy()

    # 각 클래스에 대한 확률 반환
    return probs

def analyze_post_risk(post, model, tokenizer, label_map_inv):
    # 게시글 토큰화
    tokens = tokenizer.tokenize(post)

    # 각 토큰의 위험도 예측
    probs = classify_sentence(post, model, tokenizer)

    # 각 카테고리 및 예측 확률 분석
    category_probabilities = {}
    for label, prob in zip(label_map_inv.values(), probs):
        category_probabilities[label] = prob

    # 가장 높은 확률을 가진 카테고리와 그 확률을 찾음
    max_prob_category = max(category_probabilities, key=category_probabilities.get)
    max_prob = category_probabilities[max_prob_category]

    return max_prob_category, max_prob, category_probabilities


label_map_inv = {v: k for k, v in label_map.items()}

In [ ]:
# 게시글 입력
post1 = "재배강의 문의 및 합법화 면왭 활동에 대한 아이디어나 도움 주실분, 아동성범죄 영상도 팝니다"
post2 = "페도필리아가 만든 사이트인데 사진 업로드 가능"
post3 = "웹캠으로 디폴트 암호 안바꾸고 쭉 사용하는 집은 볼 수 있었음. 부부들 잠자리 확인 가능"
post4 = "웹캠으로 찍은 몰카보고싶다"
post5 = "안녕하세요 초보농부입니다 오늘은 기분이 좋네요"

post = []
post.append(post1)
post.append(post2)
post.append(post3)
post.append(post4)
post.append(post5)

label_map_inv = {v: k for k, v in label_map.items()}

# 위험도 분석

for i in range(len(post)):
    sentence = post[i]
    max_prob_category, max_prob, category_probabilities = analyze_post_risk(sentence, model, tokenizer, label_map_inv)
    if (max_prob_category == 'normal'):
        max_prob = 1 - max_prob
    # 결과 출력
    print("입력 : ", sentence)
    print("********RISK REPORT*******")
    print(f"**위험 카테고리: {max_prob_category}\t**")
    print(f"**위험도 : {max_prob:.2f}%\t**")
    print("**각 카테고리별 확률:\t**")
    for category, prob in category_probabilities.items():
        print(f"**\t{category}: {prob:.2f}\t**")
    print("**************************")
    print()

입력 :  재배강의 문의 및 합법화 면왭 활동에 대한 아이디어나 도움 주실분, 아동성범죄 영상도 팝니다
********RISK REPORT*******
**위험 카테고리: 마약	**
**위험도 : 0.98%	**
**각 카테고리별 확률:	**
**	normal: 0.00	**
**	마약: 0.98	**
**	성범죄: 0.01	**
**	해킹: 0.00	**
**************************

입력 :  페도필리아가 만든 사이트인데 사진 업로드 가능
********RISK REPORT*******
**위험 카테고리: 성범죄	**
**위험도 : 0.95%	**
**각 카테고리별 확률:	**
**	normal: 0.00	**
**	마약: 0.03	**
**	성범죄: 0.95	**
**	해킹: 0.02	**
**************************

입력 :  웹캠으로 디폴트 암호 안바꾸고 쭉 사용하는 집은 볼 수 있었음. 부부들 잠자리 확인 가능
********RISK REPORT*******
**위험 카테고리: 해킹	**
**위험도 : 0.96%	**
**각 카테고리별 확률:	**
**	normal: 0.00	**
**	마약: 0.00	**
**	성범죄: 0.04	**
**	해킹: 0.96	**
**************************

입력 :  웹캠으로 찍은 몰카보고싶다
********RISK REPORT*******
**위험 카테고리: normal	**
**위험도 : 0.54%	**
**각 카테고리별 확률:	**
**	normal: 0.46	**
**	마약: 0.01	**
**	성범죄: 0.21	**
**	해킹: 0.31	**
**************************

입력 :  안녕하세요 초보농부입니다 오늘은 기분이 좋네요
********RISK REPORT*******
**위험 카테고리: normal	**
**위험도 : 0.39%	**
**각 카테고리별 확률:	**
**	normal: 0.61	**
**	마약: 0.38

In [ ]:
# 게시글 입력
post1= "나오늘 웹캠 털려서 유출된거 같은데 어떡하지..? 신고해야할까?"
post2= "나오늘 웹캠 털려서 유출된거 같은데 어떡하냐"

# 위험도 분석

post = []
post.append(post1)
post.append(post2)

label_map_inv = {v: k for k, v in label_map.items()}

# 위험도 분석

for i in range(len(post)):
    sentence = post[i]
    max_prob_category, max_prob, category_probabilities = analyze_post_risk(sentence, model, tokenizer, label_map_inv)
    if (max_prob_category == 'normal'):
        max_prob = 1 - max_prob
    # 결과 출력
    print("입력 : ", sentence)
    print("********RISK REPORT*******")
    print(f"**위험 카테고리: {max_prob_category}\t**")
    print(f"**위험도 : {max_prob:.2f}%\t**")
    print("**각 카테고리별 확률:\t**")
    for category, prob in category_probabilities.items():
        print(f"**\t{category}: {prob:.2f}\t**")
    print("**************************")
    print()

입력 :  나오늘 웹캠 털려서 유출된거 같은데 어떡하지..? 신고해야할까?
********RISK REPORT*******
**위험 카테고리: 성범죄	**
**위험도 : 0.81%	**
**각 카테고리별 확률:	**
**	normal: 0.00	**
**	마약: 0.04	**
**	성범죄: 0.81	**
**	해킹: 0.14	**
**************************

입력 :  나오늘 웹캠 털려서 유출된거 같은데 어떡하냐
********RISK REPORT*******
**위험 카테고리: normal	**
**위험도 : 0.14%	**
**각 카테고리별 확률:	**
**	normal: 0.86	**
**	마약: 0.01	**
**	성범죄: 0.12	**
**	해킹: 0.01	**
**************************



## 모델저장

In [ ]:
import torch

# 모델 저장 경로 지정
model_path = "/content/drive/MyDrive/KWU 4-1/TextMining/team/data/bert_ver3.pt"

# 학습된 모델 저장
torch.save(model.state_dict(), model_path)